In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

import pickle

In [6]:
#load the dataset

data=pd.read_csv('ANN/Churn_Modelling.csv')

In [7]:
#preprocess the data
#drop irrelevenat columns

data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [8]:
#categorical value to encoding
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [9]:
data['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [10]:
#one hot encoding
from sklearn.preprocessing import OneHotEncoder
one_hot_geo=OneHotEncoder()
geo_encoder=one_hot_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [11]:
one_hot_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [12]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [13]:
geo_encoder_df=pd.DataFrame(geo_encoder.toarray(),columns=one_hot_geo.get_feature_names_out(['Geography']))

In [14]:
#combine all the columns
data=pd.concat([data.drop(['Geography'],axis=1),geo_encoder_df],axis=1)

In [15]:

x=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

In [16]:

#split the data into training and testing data
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

#scale the features
Scaler=StandardScaler()
x_train=Scaler.fit_transform(x_train)
x_test=Scaler.transform(x_test)

In [18]:
# #save the encoders and scalers
with open('label_encoder_gender.pkl','wb')as file:
    pickle.dump(label_encoder_gender, file)  

with open('one_hot_geo.pkl','wb')as file:
    pickle.dump(one_hot_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(Scaler,file)

ANN Regression 

In [19]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense


In [20]:

model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1) #output layer for regression
])


e:\Krish_n_materials\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
#compile the model

model.compile(optimizer='Adam',loss='mean_absolute_error',metrics=['mae'])

In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
import datetime

log_dir="regressionlogs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callbacks=TensorBoard(log_dir=log_dir,histogram_freq=1)


In [25]:
#set up the early stoppung

early_stopping=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)


In [26]:
#train the model

history=model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100,batch_size=32,
                  callbacks=[tensorflow_callbacks,early_stopping])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - loss: 100382.9844 - mae: 100382.9844 - val_loss: 98536.2344 - val_mae: 98536.2344
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 99700.9531 - mae: 99700.9531 - val_loss: 97154.5938 - val_mae: 97154.5938
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 97294.9844 - mae: 97294.9844 - val_loss: 93608.9531 - val_mae: 93608.9531
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 92512.5156 - mae: 92512.5156 - val_loss: 87612.3281 - val_mae: 87612.3281
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 85444.3906 - mae: 85444.3906 - val_loss: 79709.5859 - val_mae: 79709.5859
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 76882.0469 - mae: 76882.0469 - val_loss: 71099.2109 - val_mae: 71099.2109
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 68262.8984 - mae: 68262.8984 - val_loss: 63393.9688 - val_mae: 63393.9688
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 1

In [28]:
#evluate the model

test_loss,test_mae=model.evaluate(x_test,y_test)
print(f'Test MAE: {test_mae}')


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 50302.5859 - mae: 50302.5859
Test MAE: 50302.5859375


In [30]:
model.save("regression.h5")